In [ ]:
from google.colab import files
files.upload()

In [ ]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d deathtrooper/multichannel-glaucoma-benchmark-dataset

In [ ]:
!unzip -qq multichannel-glaucoma-benchmark-dataset.zip

In [ ]:
import pandas as pd

dataframe = pd.read_csv("/content/metadata - standardized.csv")
dataframe.head()

In [ ]:
positive_count = sum([i for i in dataframe["types"] == 1])
negative_count = sum([i for i in dataframe["types"] == 0])
suspect_count = sum([i for i in dataframe["types"] == -1])

print(f"Number of Glaucoma positive cases in the dataset = {positive_count}")
print(f"Number if Glaucoma negative cases in the dataset = {negative_count}")
print(f"Number of Glaucoma suspect cases in the dataset = {suspect_count}")

In [ ]:
import pathlib

original_fundus_dir = pathlib.Path("full-fundus/full-fundus")

In [ ]:
orginal_dir_all_images = len([images for images in original_fundus_dir.glob("*")])
orginal_dir_all_images

In [ ]:
import os

image_path = os.path.join(original_fundus_dir, "OIA-ODIR-TEST-OFFLINE-1.png")

if os.path.exists(image_path):
    print(f"Image exists")
else:
    print(f"Image does not exist")

In [ ]:
import shutil

categorized_dir_neg = pathlib.Path("Fundus/Negative")
categorized_dir_pos = pathlib.Path("Fundus/Positive")

os.makedirs(categorized_dir_neg, exist_ok = True)
os.makedirs(categorized_dir_pos, exist_ok = True)

for index, row in dataframe.iterrows():
  src_path = "full-fundus" + row["fundus"]

  if row["types"] == 0:
    dst_path = categorized_dir_neg / pathlib.Path(row["fundus"]).name
  elif row["types"] == 1:
     dst_path = categorized_dir_pos / pathlib.Path(row["fundus"]).name

  shutil.copyfile(src=src_path, dst=dst_path)

In [ ]:
print(f"Negative count : {len([images for images in categorized_dir_neg.glob('*')])}")
print(f"Positive count : {len([images for images in categorized_dir_pos.glob('*')])}")

In [ ]:
# 2500 test, 1000 val, 1000 test

cat_dir = pathlib.Path("Fundus")
new_base_dir = pathlib.Path("Glaucoma_Fundus_Small")

def makeSubset(subset_name, start_index, end_index):
    for category in ("Negative", "Positive"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir, exist_ok=True)
        src_filenames = [filename for filename in os.listdir(cat_dir / category) if filename.endswith(".png")]
        for index, src_filenames in enumerate(src_filenames[start_index : end_index]):
          dst_filename = f"{category}.{index}.png"
          src_path = cat_dir / category / src_filenames
          dst_path = dir / dst_filename
          shutil.copyfile(src=src_path, dst=dst_path)

makeSubset("train", start_index=0, end_index=2500)
makeSubset("validation", start_index=2500, end_index=3500)
makeSubset("test", start_index=3500, end_index=4500)

In [ ]:
print(f"Test folder Negative count: {len(list((new_base_dir / 'test' / 'Negative').glob('*')))}")
print(f"Test folder Positive count: {len(list((new_base_dir / 'test' / 'Positive').glob('*')))}")

print(f"Train folder Negative count: {len(list((new_base_dir / 'train' / 'Negative').glob('*')))}")
print(f"Train folder Positive count: {len(list((new_base_dir / 'train' / 'Positive').glob('*')))}")

print(f"Validation folder Negative count: {len(list((new_base_dir / 'validation' / 'Negative').glob('*')))}")
print(f"Validation folder Positive count: {len(list((new_base_dir / 'validation' / 'Positive').glob('*')))}")

In [ ]:
import tensorflow
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.applications.inception_v3 import InceptionV3

conv_base = InceptionV3(
#  input_shape = (180, 180, 3),
 weights="imagenet",
 include_top=False)

# conv_base = keras.applications.vgg16.VGG16(
#  weights="imagenet",
#  include_top=False)

conv_base.trainable = False

In [ ]:
data_augmentation = keras.Sequential([
 layers.RandomFlip("horizontal"),
 layers.RandomRotation(0.1),
 layers.RandomZoom(0.2),
 ])

In [ ]:
# inputs = keras.Input(shape=(180, 180, 3))
# x = data_augmentation(inputs)
# x = layers.Rescaling(1./255)(x)
# x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
# x = layers.MaxPooling2D(pool_size=2)(x)
# x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
# x = layers.MaxPooling2D(pool_size=2)(x)
# x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
# x = layers.MaxPooling2D(pool_size=2)(x)
# x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
# x = layers.MaxPooling2D(pool_size=2)(x)
# x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
# x = layers.Flatten()(x)
# outputs = layers.Dense(1, activation="sigmoid")(x)

# inputs = keras.Input(shape=(180, 180, 3))
# x = data_augmentation(inputs)
# x = layers.Rescaling(1./255)(x)
# x = keras.applications.vgg16.preprocess_input(x)
# x = conv_base(x)
# x = layers.Flatten()(x)
# x = layers.Dense(256)(x)
# x = layers.Dropout(0.5)(x)
# outputs = layers.Dense(1, activation="sigmoid")(x)
# model = keras.Model(inputs=inputs, outputs=outputs)

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
 new_base_dir / "train",
 image_size=(180, 180),
 batch_size=50)

validation_dataset = image_dataset_from_directory(
 new_base_dir / "validation",
 image_size=(180, 180),
 batch_size=50)

test_dataset = image_dataset_from_directory(
 new_base_dir / "test",
 image_size=(180, 180),
 batch_size=50)

In [ ]:
callbacks = [
 keras.callbacks.ModelCheckpoint(
 filepath="feature_extraction_with_data_augmentation.keras",
 save_best_only=True,
 monitor="val_loss")
]

In [ ]:
history = model.fit(
 train_dataset,
 epochs=30,
 validation_data=validation_dataset,
 callbacks=callbacks)

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

In [ ]:
test_model = keras.models.load_model("feature_extraction_with_data_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")